In [1]:
import os
import csv
import json
import shutil
import pandas as pd
from tqdm import tqdm
import numpy as np
from PIL import Image
from sklearn.model_selection import KFold
from PIL import Image, UnidentifiedImageError
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit

IF IMAGES ARE .WEBP RUN BELOW

In [2]:
def convert_webp_to_png(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.webp'):
                webp_path = os.path.join(subdir, file)
                png_path = os.path.join(subdir, os.path.splitext(file)[0] + '.png')
                
                # Open the WEBP and convert it to PNG
                image = Image.open(webp_path)
                image.save(png_path, 'PNG')
                print(f"Converted {webp_path} to {png_path}")

                # Optionally, remove the original WEBP file
                os.remove(webp_path)
                print(f"Removed original {webp_path}")

# Define your base directory
base_dir = "/home/lkandakji/image_analysis/segmentation/nnunet/all_annotations"

# Convert all WEBP images in the base directory
convert_webp_to_png(base_dir)

Converted /home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/76ad13b7-5b50-4e2e-8db0-b964cbba3881-000000/326/Feature A/images/76ad13b7-5b50-4e2e-8db0-b964cbba3881_11.webp to /home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/76ad13b7-5b50-4e2e-8db0-b964cbba3881-000000/326/Feature A/images/76ad13b7-5b50-4e2e-8db0-b964cbba3881_11.png
Removed original /home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/76ad13b7-5b50-4e2e-8db0-b964cbba3881-000000/326/Feature A/images/76ad13b7-5b50-4e2e-8db0-b964cbba3881_11.webp
Converted /home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/76ad13b7-5b50-4e2e-8db0-b964cbba3881-000000/326/Feature A/images/76ad13b7-5b50-4e2e-8db0-b964cbba3881_00.webp to /home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/76ad13b7-5b50-4e2e-8db0-b964cbba3881-000000/326/Feature A/images/76ad13b7-5b50-4e2e-8db0-b964cbba3881_00.png
Removed original /home/lkandakji/image_analysis/segmentation/nnunet/all_an

IF IMAGES ARE .BMP RUN BELOW

In [12]:
def convert_bmp_to_png(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.bmp'):
                bmp_path = os.path.join(subdir, file)
                png_path = os.path.join(subdir, os.path.splitext(file)[0] + '.png')
                
                # Open and convert the image
                image = Image.open(bmp_path)
                image.save(png_path, 'PNG')
                print(f"Converted {bmp_path} to {png_path}")

                # Optionally remove the original
                os.remove(bmp_path)
                print(f"Removed original {bmp_path}")

# Define the two image directories
dirs_to_convert = [
    "/home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Full-frame_Dataset/Original_AS-OCT_Images",
    "/home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Partial-frame_Dataset/Original_AS-OCT_Images"
]

# Run conversion on both
for d in dirs_to_convert:
    convert_bmp_to_png(d)

CREATE CSV

In [13]:
base_dir = "/home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset"
csv_file_path = os.path.join(base_dir, "image_label_mapping.csv")

with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['dataset_type', 'image_path', 'label_path'])

    dataset_types = ['Full-frame_Dataset', 'Partial-frame_Dataset']
    for dtype in dataset_types:
        image_dir = os.path.join(base_dir, dtype, "Original_AS-OCT_Images")
        label_dir = os.path.join(base_dir, dtype, "annotations")

        if os.path.exists(image_dir) and os.path.exists(label_dir):
            images = sorted(os.listdir(image_dir))
            labels = sorted(os.listdir(label_dir))

            for img, lbl in zip(images, labels):
                if img == lbl:
                    img_path = os.path.join(image_dir, img)
                    lbl_path = os.path.join(label_dir, lbl)
                    writer.writerow([dtype, img_path, lbl_path])
                else:
                    print(f"[{dtype}] Mismatch: {img} ≠ {lbl}")
        else:
            print(f"Missing directory: {image_dir} or {label_dir}")

print(f"CSV file created at {csv_file_path}")

Missing directory: /home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Full-frame_Dataset/Original_AS-OCT_Images or /home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Full-frame_Dataset/annotations
Missing directory: /home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Partial-frame_Dataset/Original_AS-OCT_Images or /home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Partial-frame_Dataset/annotations
CSV file created at /home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/image_label_mapping.csv


In [14]:
base_csv_path = "/home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/image_label_mapping.csv"
new_data_csv_path = "/mnt/data/MS39_Raw/CSO2/stage1_segmentation.csv"

with open(base_csv_path, 'r', newline='') as file:
    reader = csv.reader(file)
    existing_data = list(reader)

if 'condition' not in existing_data[0]:
    existing_data[0].append('condition')

with open(new_data_csv_path, 'r', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if row['filename'] != "OCT-0_23.png":
            new_row = [row['vol_id'], row['scan'], row['label'], "keratoconus"]
            existing_data.append(new_row)

with open(base_csv_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(existing_data)

print(f"CSV file updated and saved at {base_csv_path}")

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/MS39_Raw/CSO2/stage1_segmentation.csv'

In [ ]:
new_data_csv_path = "/mnt/data/MS39_Raw/CSO2/stage4_segmentation.csv"

with open(base_csv_path, 'r', newline='') as file:
    reader = csv.reader(file)
    existing_data = list(reader)

with open(new_data_csv_path, 'r', newline='') as file:
    reader = csv.DictReader(file)
    for row in reader:
        if row['filename'] != "OCT-0_23.png":
            new_row = [row['vol_id'], row['scan'], row['label'], "keratoconus"]
            existing_data.append(new_row)

with open(base_csv_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(existing_data)

print(f"CSV file updated and saved at {base_csv_path}")

CSV file updated and saved at /home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/image_label_mapping.csv


In [ ]:
condition_files = {
    "fecd": "/mnt/scratch/mehphoenix/fecd_seg.csv",
    "keratoconus": "/mnt/scratch/mehphoenix/kc_seg.csv",
    "normal": "/mnt/scratch/mehphoenix/normals_seg.csv"
}

patient_conditions = {}

for condition, path in condition_files.items():
    with open(path, 'r', newline='') as file:
        reader = csv.reader(file)
        for row in reader:
            patient_id = row[-1].split('/')[-1]
            patient_conditions[patient_id] = condition

updated_data = []
with open(base_csv_path, 'r', newline='') as file:
    reader = csv.reader(file)
    headers = next(reader) 
    if 'condition' not in headers:
        headers.append('condition') 
    updated_data.append(headers)

    for row in reader:
        patient_id = row[0]
        if patient_id in patient_conditions:
            if len(row) == len(headers):
                row[-1] = patient_conditions[patient_id]
            else:
                row.append(patient_conditions[patient_id])
        updated_data.append(row)

with open(base_csv_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(updated_data)

print(f"CSV file updated and saved at {base_csv_path}")

CSV file updated and saved at /home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/image_label_mapping.csv


In [ ]:
base_csv_path = "/home/lkandakji/image_analysis/segmentation/nnunet/all_annotations/image_label_mapping.csv"

In [ ]:
data = pd.read_csv(base_csv_path)
train_csv_path = '/home/lkandakji/image_analysis/segmentation/nnunet/train_kera.csv'
val_csv_path = '/home/lkandakji/image_analysis/segmentation/nnunet/val_kera.csv'
test_csv_path = '/home/lkandakji/image_analysis/segmentation/nnunet/test_kera.csv'

data['patient_condition'] = data['patient'] + "_" + data['condition']

gss = GroupShuffleSplit(n_splits=1, test_size=0.30, random_state=42)

for train_idx, test_val_idx in gss.split(data, groups=data['patient_condition']):
    train_data = data.iloc[train_idx]
    test_val_data = data.iloc[test_val_idx]

gss_val_test = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for val_idx, test_idx in gss_val_test.split(test_val_data, groups=test_val_data['patient_condition']):
    val_data = test_val_data.iloc[val_idx]
    test_data = test_val_data.iloc[test_idx]

train_data.drop('patient_condition', axis=1, inplace=True)
val_data.drop('patient_condition', axis=1, inplace=True)
test_data.drop('patient_condition', axis=1, inplace=True)

train_data.to_csv(train_csv_path, index=False)
val_data.to_csv(val_csv_path, index=False)
test_data.to_csv(test_csv_path, index=False)

print("Data split into training, validation, and test sets successfully with no patient overlap.")

Data split into training, validation, and test sets successfully with no patient overlap.


/tmp/ipykernel_1602177/3132287476.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop('patient_condition', axis=1, inplace=True)
/tmp/ipykernel_1602177/3132287476.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data.drop('patient_condition', axis=1, inplace=True)
/tmp/ipykernel_1602177/3132287476.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop('patient_condition', axis=1, inplace=True)


In [ ]:
def print_unique_counts(csv_path, set_name):
    data = pd.read_csv(csv_path)
    num_patients = data['patient'].nunique()
    condition_counts = data.groupby('condition')['patient'].nunique()
    print(f"{set_name} set has {num_patients} unique patients and {condition_counts}.")

print_unique_counts(train_csv_path, "Training")
print_unique_counts(val_csv_path, "Validation")
print_unique_counts(test_csv_path, "Test")

Training set has 209 unique patients and condition
fecd           71
keratoconus    67
normal         71
Name: patient, dtype: int64.
Validation set has 45 unique patients and condition
fecd           16
keratoconus    18
normal         11
Name: patient, dtype: int64.
Test set has 45 unique patients and condition
fecd           13
keratoconus    15
normal         17
Name: patient, dtype: int64.


In [15]:
image_dir = "/home/lkandakji/image_analysis/segmentation/nnunet/sclera_lens_photos"
csv_file_path = os.path.join(image_dir, "scleral_photo_image_mapping.csv")

image_filenames = ["irregular_cornea.png", "regular_cornea.png"]

with open(csv_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['dataset_type', 'image_path', 'label_path'])

    for img in image_filenames:
        img_path = os.path.join(image_dir, img)
        if os.path.exists(img_path):
            writer.writerow(["scleral_lens_photo", img_path, ""])
        else:
            print(f"Image not found: {img_path}")

print(f"CSV file created at {csv_file_path}")

CSV file created at /home/lkandakji/image_analysis/segmentation/nnunet/sclera_lens_photos/scleral_photo_image_mapping.csv


NN-UNET PREP

In [ ]:
# Set test_out to True for testing with 100 pairs, False for full processing
test_out = False
NUM_TEST_OUT = 1000

#REPLACE PATHS
train_csv_path = '/home/lkandakji/image_analysis/segmentation/nnunet/train_kera.csv'
val_csv_path = '/home/lkandakji/image_analysis/segmentation/nnunet/val_kera.csv'
test_csv_path = '/home/lkandakji/image_analysis/segmentation/nnunet/test_kera.csv'
output_base_path = '/home/lkandakji/image_analysis/segmentation/nnunet/dataset/nnunet_raw/nnunet_raw_data/Dataset001_kera'
name_mapping_csv = '/home/lkandakji/image_analysis/segmentation/nnunet/nnunet_name_mapping_kera.csv'

In [ ]:
output_base_path = "/home/lkandakji/image_analysis/segmentation/nnunet/dataset/nnunet_raw/nnunet_raw_data/Dataset001_external"

In [17]:
output_base_path = "/home/lkandakji/image_analysis/segmentation/nnunet/dataset/nnunet_raw/nnunet_raw_data/Dataset001_scleral"
os.makedirs(os.path.join(output_base_path, 'imagesTs'), exist_ok=True)


In [ ]:
# Create directories
os.makedirs(os.path.join(output_base_path, 'imagesTr'), exist_ok=True)
os.makedirs(os.path.join(output_base_path, 'labelsTr'), exist_ok=True)
os.makedirs(os.path.join(output_base_path, 'imagesTs'), exist_ok=True)
os.makedirs(os.path.join(output_base_path, 'labelsTs'), exist_ok=True)
os.makedirs(os.path.join(output_base_path, 'visTr'), exist_ok=True)
os.makedirs(os.path.join(output_base_path, 'visTs'), exist_ok=True)

In [ ]:
# Read CSV files
train_df = pd.read_csv(train_csv_path)
val_df = pd.read_csv(val_csv_path)
test_df = pd.read_csv(test_csv_path)

# Combine train and validation data
combined_df = pd.concat([train_df, val_df])
combined_df.head()

,patient,image_path,label_path,condition
0,7e38d484-a2b7-43ca-a0f8-90f455c3266c,/home/lkandakji/image_analysis/segmentation/nn...,/home/lkandakji/image_analysis/segmentation/nn...,keratoconus
1,7e38d484-a2b7-43ca-a0f8-90f455c3266c,/home/lkandakji/image_analysis/segmentation/nn...,/home/lkandakji/image_analysis/segmentation/nn...,keratoconus
2,31ecd98a-767c-4a83-8c59-95057cd6f3b2,/home/lkandakji/image_analysis/segmentation/nn...,/home/lkandakji/image_analysis/segmentation/nn...,fecd
3,31ecd98a-767c-4a83-8c59-95057cd6f3b2,/home/lkandakji/image_analysis/segmentation/nn...,/home/lkandakji/image_analysis/segmentation/nn...,fecd
4,d578e9ee-e3cb-4e04-a8e7-de20c2ba1b40,/home/lkandakji/image_analysis/segmentation/nn...,/home/lkandakji/image_analysis/segmentation/nn...,normal


In [ ]:
# Define label mapping
label_mapping = {'label_path': 1}

def visualize_and_save(image_path, label_path, output_path):
    # Load the image and label
    image = Image.open(image_path)
    label = Image.open(label_path)

    # Convert label to RGB for visualization
    label_array = np.array(label)
    label_rgb = np.zeros((*label_array.shape, 3), dtype=np.uint8)  # Initialize RGB array

    # Assign colors to each class (excluding background)
    colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]  # Red, Green, Blue, Yellow for 4 classes
    for i, color in enumerate(colors, start=1):
        label_rgb[label_array == i] = color

    label_rgb_image = Image.fromarray(label_rgb)

    # Overlay label on image
    combined = Image.blend(image.convert('RGBA'), label_rgb_image.convert('RGBA'), alpha=0.5)

    # Save the visualization
    combined.save(output_path)

# Function to process and copy images
def process_and_copy(row, index, is_test=False):
    try:
        input_image_path = row['image_path']
        pid = row['patient']
        new_name = f'kera_{index:05d}'

        # Open the input image and convert to grayscale if necessary
        input_image = Image.open(input_image_path)
        if input_image.mode != 'L':  # 'L' mode is for grayscale images
            input_image = input_image.convert('L')

        output_image_path = os.path.join(output_base_path, 'imagesTs' if is_test else 'imagesTr', new_name + '_0000.png')
        input_image.save(output_image_path)

        # Initialize label_image with the size of the input image
        label_image = np.zeros(input_image.size, dtype=np.uint8)

        for mask, label_index in label_mapping.items():
            if row[mask] != 'blank':
                annotation = Image.open(row[mask]).convert('L')  # Convert to grayscale
                annotation_array = np.array(annotation)
                
                # Resize label_image if dimensions don't match
                if annotation_array.shape != label_image.shape:
                    label_image = np.zeros(annotation_array.shape, dtype=np.uint8)

                label_image[annotation_array > 0] = label_index
                
        label_image_path = os.path.join(output_base_path, 'labelsTs' if is_test else 'labelsTr', new_name + '.png')
        Image.fromarray(label_image).save(label_image_path)

        # Visualization (for a subset of images)
        if index < 10:
            vis_output_path = os.path.join(output_base_path, 'visTs' if is_test else 'visTr', new_name + '.png')
            visualize_and_save(output_image_path, label_image_path, vis_output_path)

        return input_image_path, new_name

    except (IOError, FileNotFoundError, UnidentifiedImageError) as e:
        print(f"Error processing file: {input_image_path}. Error: {e}. Skipping.")
        return None, None

In [ ]:
# Input directories
input_dirs = [
    "/home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Full-frame_Dataset/Original_AS-OCT_Images",
    "/home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Partial-frame_Dataset/Original_AS-OCT_Images"
]

# Output directories
output_base_path = "/home/lkandakji/image_analysis/segmentation/nnunet/dataset/nnunet_raw/nnunet_raw_data/Dataset001_external"
output_image_dir = os.path.join(output_base_path, "imagesTs")

# Ensure output directories exist
os.makedirs(output_image_dir, exist_ok=True)

In [20]:
input_dir = "/home/lkandakji/image_analysis/segmentation/nnunet/sclera_lens_photos"

In [21]:
def visualize_and_save(image_path, output_path):
    image = Image.open(image_path).convert('L')  # Ensure grayscale
    image.save(output_path)
    
def process_and_copy():
    index = 0

    for input_dir in input_dirs:
        for root, _, files in os.walk(input_dir):
            for file in sorted(files):
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                    image_path = os.path.join(root, file)
                    new_name = f'kera_{index:05d}_0000.png'
                    output_image_path = os.path.join(output_image_dir, new_name)

                    try:
                        img = Image.open(image_path)
                        if img.mode != 'L':
                            img = img.convert('L')
                        img.save(output_image_path)

                    except (IOError, FileNotFoundError, UnidentifiedImageError) as e:
                        print(f"Error processing {image_path}: {e}")
                        continue

In [22]:
# Process images
name_mapping = []
index = 0
for _, row in tqdm(combined_df.iterrows(), total=combined_df.shape[0]):
    original_path, new_name = process_and_copy(row, index)
    if original_path is not None:
        name_mapping.append([original_path, new_name])
        index += 1
        if test_out and index >= 100:
            break

# Process test images
for _, row in tqdm(test_df.iterrows(), total=test_df.shape[0]):
    original_path, new_name = process_and_copy(row, index, is_test=True)
    name_mapping.append([original_path, new_name])
    index += 1
    if test_out and index >= NUM_TEST_OUT:
        break

# Save name mapping
pd.DataFrame(name_mapping, columns=['OriginalPath', 'NNUNetName']).to_csv(name_mapping_csv, index=False)

NameError: name 'combined_df' is not defined

In [ ]:
name_mapping = []
index = 0

# Process all images
for input_dir in input_dirs:
    for root, _, files in os.walk(input_dir):
        for file in tqdm(sorted(files), desc=f"Processing {input_dir}"):
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
                image_path = os.path.join(root, file)
                new_name = f'kera_{index:05d}_0000.png'
                output_path = os.path.join(output_image_dir, new_name)

                try:
                    img = Image.open(image_path)
                    if img.mode != 'L':
                        img = img.convert('L')
                    img.save(output_path)

                    name_mapping.append([image_path, new_name])
                    index += 1

                except (IOError, FileNotFoundError, UnidentifiedImageError) as e:
                    print(f"Error processing {image_path}: {e}")

# Save mapping CSV
mapping_csv_path = os.path.join(os.path.dirname(output_image_dir), "image_name_mapping.csv")
pd.DataFrame(name_mapping, columns=['OriginalPath', 'NNUNetName']).to_csv(mapping_csv_path, index=False)

print(f"Done. Saved {index} images.")
print(f"Name mapping saved to: {mapping_csv_path}")

Processing /home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Full-frame_Dataset/Original_AS-OCT_Images: 100%|██████████| 769/769 [01:43<00:00,  7.46it/s]
Processing /home/lkandakji/image_analysis/segmentation/nnunet/AIDK_Dataset/Partial-frame_Dataset/Original_AS-OCT_Images: 100%|██████████| 401/401 [00:53<00:00,  7.44it/s]

Done. Saved 1168 images.
Name mapping saved to: /home/lkandakji/image_analysis/segmentation/nnunet/dataset/nnunet_raw/nnunet_raw_data/Dataset001_external/image_name_mapping.csv


In [3]:
# Paths
input_dir = "/home/lkandakji/image_analysis/segmentation/nnunet/sclera_lens_photos"
output_base_path = "/home/lkandakji/image_analysis/segmentation/nnunet/dataset/nnunet_raw/nnunet_raw_data/Dataset001_scleral"
output_image_dir = os.path.join(output_base_path, 'imagesTs')

# Initialize
image_files = ["irregular_cornea.png", "regular_cornea.png"]
name_mapping = []

# Process each image
for index, file in enumerate(image_files):
    image_path = os.path.join(input_dir, file)
    new_name = f'kera_{index:05d}_0000.png'
    output_path = os.path.join(output_image_dir, new_name)

    try:
        img = Image.open(image_path)
        if img.mode != 'L':
            img = img.convert('L')
        img.save(output_path)
        name_mapping.append([image_path, new_name])

    except (IOError, FileNotFoundError, UnidentifiedImageError) as e:
        print(f"Error processing {image_path}: {e}")

# Save name mapping CSV
mapping_csv_path = os.path.join(output_base_path, "image_name_mapping.csv")
pd.DataFrame(name_mapping, columns=['OriginalPath', 'NNUNetName']).to_csv(mapping_csv_path, index=False)

print(f"✅ Done. Saved {len(name_mapping)} images.")
print(f"📄 Name mapping saved to: {mapping_csv_path}")

✅ Done. Saved 2 images.
📄 Name mapping saved to: /home/lkandakji/image_analysis/segmentation/nnunet/dataset/nnunet_raw/nnunet_raw_data/Dataset001_scleral/image_name_mapping.csv


In [ ]:
# Create dataset.json
dataset_json = {
    "channel_names": {"0": "0000"},
    "labels": {"background": 0, "label_path": 1},
    "numTraining": index if not test_out else NUM_TEST_OUT,
    "file_ending": ".png"
}
with open(os.path.join(output_base_path, 'dataset.json'), 'w') as f:
    json.dump(dataset_json, f, indent=4)

NameError: name 'test_out' is not defined

In [4]:
# Create dataset.json
dataset_json = {
    "channel_names": {"0": "0000"},
    "labels": {},  # No training, so no labels
    "numTraining": 0,
    "file_ending": ".png",
}
with open(os.path.join(output_base_path, 'dataset.json'), 'w') as f:
    json.dump(dataset_json, f, indent=4)

In [ ]:
def get_file_identifiers(directory):
    # Extracts file identifiers from the filenames (assumes format 'neo_{index}_0000.png')
    files = os.listdir(directory)
    identifiers = set(os.path.splitext(file)[0].split('_')[1] for file in files if file.endswith('.png'))
    return identifiers

# Get identifiers from imagesTr and labelsTr
images_identifiers = get_file_identifiers(os.path.join(output_base_path, 'imagesTr'))
labels_identifiers = get_file_identifiers(os.path.join(output_base_path, 'labelsTr'))

# Ensure that each image has a corresponding label
valid_identifiers = images_identifiers.intersection(labels_identifiers)

# Convert to list and sort
valid_identifiers = sorted(list(valid_identifiers))

In [ ]:
# Create 5-fold validation split considering patient IDs
group_kfold = GroupKFold(n_splits=5)
splits = []

# Use 'pid' column from combined_df as the groups
for train_index, val_index in group_kfold.split(combined_df, groups=combined_df['patient']):
    train_ids = [f'kera_{combined_df.iloc[i].name:05d}' for i in train_index]
    val_ids = [f'kera_{combined_df.iloc[i].name:05d}' for i in val_index]
    splits.append({'train': train_ids, 'val': val_ids})

# Save splits
with open('/home/lkandakji/image_analysis/segmentation/nnunet/dataset/nnunet_preprocessed/splits_final.json', 'w') as f:
    json.dump(splits, f, indent=4)

Note: change training cases value in dataset.json